#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

import datetime


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)


# Добавление строки в лог
def add_log(sname):
    f = open('log.csv','a')
    f.write(str(sname)+','+ datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n')
    f.close()
    return True

server.register_function(add_log, 'add_log')

def get_log(event_type, start_time, end_time):
    filtered_events = []
    with open('event_log.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            timestamp, event = row
            timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
            if event == event_type and start_time <= timestamp <= end_time:
                filtered_events.append(row)
    return filtered_events

server.register_function(get_log, 'get_log')

print ("Listening on port 8018...")
server.serve_forever()




def logs_slice_by_time2(left, right, operation):
    date_pattern = '%Y-%m-%d %H:%M:%S'
    left_date = datetime.datetime.strptime(left, date_pattern)
    right_date = datetime.datetime.strptime(right, date_pattern)
    csv_files_names = list(filter(lambda x: x.endswith('.csv'), os.listdir(os.getcwd())))
    csv_files_dates = list(map(lambda x: x[0:len(x) - 4], csv_files_names))
    csv_files_parsed_dates = list(
        map(
            lambda d: datetime.datetime.strptime(d, file_date_pattern),
            list(filter(lambda x: x != 'log' and x != 'new_file_name', csv_files_dates))
        )
    )

    csv_files_names.remove('log.csv')
    left_index = 0
    right_index = len(csv_files_names)
    for i in range(len(csv_files_parsed_dates) - 1):
        if csv_files_parsed_dates[i] < left_date >= csv_files_parsed_dates[i + 1]:
            left_index = i + 1
        if csv_files_parsed_dates[i] < right_date <= csv_files_parsed_dates[i + 1]:
            right_index = i + 1

    csv_files_names_in_range = csv_files_names[left_index:right_index]
    csv_files = list(map(lambda x: open(x, encoding='utf8'), csv_files_names_in_range))
    logs_from_files = files_to_logs(csv_files)
    if operation is None:
        return logs_from_files
    else:
        result = list()
        for log_line in logs_from_files:
            if operator.eq(log_line[1], operation):
                result.append(log_line)
        return result
server.register_function(logs_slice_by_time2, 'logs_slice_by_time2')


def files_to_logs(csv_files):
    result = list()
    for f in csv_files:
        reader = csv.reader(f)
        for line in reader:
            result.append(line)
        f.close()

    return result


Listening on port 8018...


127.0.0.1 - - [04/Dec/2023 21:04:59] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2023 21:05:51] "POST /RPC2 HTTP/1.1" 200 -
